<a href="https://www.kaggle.com/code/ryancardwell/orcastratorv1-1?scriptVersionId=271481648" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Cell 1: Imports and Configuration
import os, sys, json, time, math, random, logging
from pathlib import Path
from collections import defaultdict, Counter, deque, OrderedDict
from typing import List, Dict, Tuple, Optional
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass, field
from scipy.optimize import linear_sum_assignment
from scipy.ndimage import label, binary_fill_holes, binary_erosion, binary_dilation
from multiprocessing import Pool
from torch.distributions import Categorical
import cv2
import scipy as sp

print("=== CELL 1: IMPORTS AND CONFIGURATION ===")
print("WHO: System initialization")
print("WHAT: Loading all required libraries and setting up configuration")
print("WHEN: At notebook startup")
print("WHERE: In memory and GPU if available")
print("WHY: To provide all necessary dependencies for ARC solving")
print("HOW: Importing standard ML, computer vision, and math libraries")

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)-7s | %(message)s")
log = logging.getLogger("ChronosOrca")

CONFIG = {
    "SEED": 1337,
    "TIME_PER_TASK": 8.0,
    "BEAM_WIDTH": 16,
    "BEAM_DEPTH": 6,
    "FAMILY_CAPS": {"geometric": 2, "color": 3, "size": 2, "object": 3, "pattern": 2},
    "RELAXED_OK": True,
    "RELAXED_MIN_IOU": 0.90,
    "BG": 0,
    "VETO_THRESH": 0.35,
    "VETO_FEATS": 32,
    "VETO_HIDDEN": 64,
    "TTT_STEPS": 5,
    "TTT_LR": 1e-3,
    "MAX_PERIOD": 12,
    "MAX_GRID_SIDE": 60,
    "ATTEMPTS_PER_TEST": 2,
    "PRINT_SAMPLE_GRIDS": False,
    "METRIC_FLUSH_EVERY": 1,
    "PERIOD_TOL": 2,
    "CACHE_CAPACITY": 1000,
    "TIME_BUDGET_S": 30.0,
    "VETO_ENABLED": True,
    "EARLY_STOP_CONF": 0.95,
    "CONFIDENCE_THRESHOLD": 0.8,
    "RELAXED_FIT_OK": True,
    "LATENT_DIM": 32,
    "POLICY_DEPTH": 2,
    "POLICY_HEADS": 4,
    "NUM_WORKERS": 2,
    "HYPER_DIM": 128
}

# Set seeds for reproducibility
random.seed(CONFIG["SEED"])
np.random.seed(CONFIG["SEED"])
torch.manual_seed(CONFIG["SEED"])

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✓ Device set to: {device}")
print(f"✓ Configuration loaded with {len(CONFIG)} parameters")
print(f"✓ Random seeds initialized with seed {CONFIG['SEED']}")

class MetricLog:
    def __init__(self):
        self.rows = []
        self.start = time.time()
        print("✓ MetricLog initialized - tracking performance metrics")
    
    def log(self, **kwargs):
        row = {"t": round(time.time() - self.start, 3)}
        row.update(kwargs)
        self.rows.append(row)
        msg = " | ".join(f"{k}={v}" for k,v in row.items())
        log.info(f"[metrics] {msg}")
    
    def summary(self):
        agg = defaultdict(list)
        for r in self.rows:
            for k,v in r.items():
                if k == "t": continue
                try:
                    agg[k].append(float(v))
                except:
                    pass
        return {k: round(sum(v)/max(1,len(v)),4) for k,v in agg.items()}

METRICS = MetricLog()

# Utility functions
def grid_to_np(g):
    if not g or not isinstance(g[0], list): 
        return np.zeros((1,1), dtype=np.uint8)
    arr = np.array(g, dtype=np.uint8)
    arr[arr>9]=9; arr[arr<0]=0
    return arr

def np_to_grid(a):
    a = np.asarray(a, dtype=np.int16)
    a[a>9]=9; a[a<0]=0
    return a.astype(int).tolist()

def same_shape(a,b):
    return a.shape == b.shape

def safe_iou(a,b):
    if not same_shape(a,b): return 0.0
    return float((a==b).sum())/max(1,a.size)

def palette(g):
    return sorted(list(np.unique(g)))

print("✓ All utility functions defined (grid conversion, metrics, etc.)")
print("=== CELL 1 COMPLETED SUCCESSFULLY ===\n")

=== CELL 1: IMPORTS AND CONFIGURATION ===
WHO: System initialization
WHAT: Loading all required libraries and setting up configuration
WHEN: At notebook startup
WHERE: In memory and GPU if available
WHY: To provide all necessary dependencies for ARC solving
HOW: Importing standard ML, computer vision, and math libraries
✓ Device set to: cpu
✓ Configuration loaded with 30 parameters
✓ Random seeds initialized with seed 1337
✓ MetricLog initialized - tracking performance metrics
✓ All utility functions defined (grid conversion, metrics, etc.)
=== CELL 1 COMPLETED SUCCESSFULLY ===



In [2]:
# Cell 2: Utilities and Grid Primitives
print("=== CELL 2: UTILITIES AND GRID PRIMITIVES ===")
print("WHO: Core system components")
print("WHAT: Defining budget management, data loading, and grid operations")
print("WHEN: During system initialization")
print("WHERE: In memory for runtime operations")
print("WHY: To handle ARC data and provide fundamental grid manipulation")
print("HOW: Implementing classes for data management and grid processing")

class NeuroBudget:
    def __init__(self, total_steps=24000, task_steps=200, reserve=0.1):
        self.total = total_steps
        self.task = task_steps
        self.reserve = reserve
        self.used = 0
        print(f"✓ NeuroBudget initialized: {total_steps} total steps, {reserve*100}% reserve")
    
    def estimate_cost(self, H, W, comp, mult=1.0):
        cost = int(max(1, math.log(H*W+1)*2) * math.log(comp+1)*1.5 * mult)
        return cost
    
    def allow(self, cost, expensive=False):
        if expensive and self.used > self.total * (1 - self.reserve): 
            return False
        return self.used + cost <= self.total
    
    def charge(self, cost):
        self.used += cost

class ARCDataLoader:
    def __init__(self, root="/kaggle/input/arc-prize-2024"):
        self.root = Path(root)
        self.files = {
            'train_ch': "arc-agi_training_challenges.json",
            'train_sol': "arc-agi_training_solutions.json",
            'eval_ch': "arc-agi_evaluation_challenges.json",
            'eval_sol': "arc-agi_evaluation_solutions.json",
            'test_ch': "arc-agi_test_challenges.json"
        }
        print(f"✓ ARCDataLoader initialized with root: {root}")
    
    def load_tasks(self, type='test'):
        file_key = f'{type}_ch'
        if file_key not in self.files:
            print(f"✗ File type '{type}' not found in available files")
            return {}
        
        path = self.root / self.files[file_key]
        if not path.exists():
            print(f"✗ File not found: {path}")
            return {}
            
        with open(path, 'r') as f:
            data = json.load(f)
        tasks = {id: {'train': d.get('train', []), 'test': d.get('test', [])} for id, d in data.items()}
        print(f"✓ Loaded {len(tasks)} {type} tasks from {path}")
        return tasks
    
    def load_solutions(self, type='train'):
        file_key = f'{type}_sol'
        if file_key not in self.files:
            print(f"✗ Solution file type '{type}' not available")
            return {}
        
        path = self.root / self.files[file_key]
        if not path.exists():
            print(f"✗ Solution file not found: {path}")
            return {}
            
        with open(path, 'r') as f:
            data = json.load(f)
        print(f"✓ Loaded {len(data)} {type} solutions from {path}")
        return data

class GridPrimitives:
    @staticmethod
    def clamp_grid(G):
        if not G: return [[0]]
        H = len(G); W = max(len(r) if isinstance(r, list) else 0 for r in G) or 1
        clamped = [[max(0, min(9, int(x))) for x in (row + [0]*W)[:W]] for row in G]
        return clamped
    
    @staticmethod
    def grid_size(G):
        G = GridPrimitives.clamp_grid(G)
        return len(G), len(G[0]) if G else 0
    
    @staticmethod
    def mode_color(G):
        flat = [x for r in G for x in r]
        return Counter(flat).most_common(1)[0][0] if flat else 0
    
    @staticmethod
    def rotate_90(G):
        G = GridPrimitives.clamp_grid(G)
        H, W = GridPrimitives.grid_size(G)
        rotated = [[G[H-1-r][c] for r in range(H)] for c in range(W)]
        return rotated
    
    @staticmethod
    def connected_components(G, bg_color=CONFIG["BG"]):
        G = np.array(GridPrimitives.clamp_grid(G))
        H, W = G.shape
        components = []
        for color in range(10):
            if color == bg_color: continue
            mask = (G == color)
            labeled, num_features = label(mask)
            for fid in range(1, num_features + 1):
                feature_mask = (labeled == fid)
                if feature_mask.sum() > 0:
                    components.append((feature_mask, color))
        print(f"✓ Found {len(components)} connected components in grid {G.shape}")
        return components

    @staticmethod
    def discover_period(G, axis=1):
        G = np.array(GridPrimitives.clamp_grid(G))
        H, W = G.shape
        max_period = CONFIG["MAX_PERIOD"]
        tol = CONFIG["PERIOD_TOL"]
        
        if axis == 1:  # horizontal
            for p in range(1, min(W, max_period) + 1):
                tile = np.tile(G[:, :p], (1, W // p + 1))[:, :W]
                mismatches = np.sum(G != tile)
                if mismatches <= tol:
                    print(f"✓ Discovered horizontal period: {p}")
                    return p
        else:  # vertical
            for p in range(1, min(H, max_period) + 1):
                tile = np.tile(G[:p, :], (H // p + 1, 1))[:H, :]
                mismatches = np.sum(G != tile)
                if mismatches <= tol:
                    print(f"✓ Discovered vertical period: {p}")
                    return p
        print("✗ No period discovered")
        return None

def bioelectric_ode(grid, t=1.0):
    if np.size(grid) == 0: return grid
    try:
        x = sp.symbols('x')
        ode = sp.diff(x, t) + x - float(np.mean(grid))
        sol = sp.dsolve(ode)
        result = np.full_like(grid, float(sol.rhs.subs(t, 0.5)), dtype=np.float32)
        print("✓ Bioelectric ODE computed successfully")
        return result
    except Exception as e:
        print(f"✗ Bioelectric ODE failed: {e}")
        return grid

def orch_or_noise(tensor, scale=0.01):
    noisy = tensor + scale * torch.randn_like(tensor) * (tensor > 0).float()
    print("✓ Applied orchestrated noise to tensor")
    return noisy

def calibrate_confidence(raw_confidence: float, method: str, program_length: int = 0, from_cache: bool = False) -> float:
    conf = raw_confidence
    if method == "exact_rule":
        conf *= 1.1
    elif method == "beam_search":
        penalty = 0.95 ** max(0, program_length - 2)
        conf *= penalty
    elif method == "neural":
        conf *= 0.9
    elif method == "hybrid_verified":
        conf *= 1.2
    
    if from_cache:
        conf = min(1.0, conf * 1.05)
    
    calibrated = min(1.0, conf)
    print(f"✓ Confidence calibrated: {raw_confidence:.3f} -> {calibrated:.3f} via {method}")
    return calibrated

print("✓ All utility classes and functions defined successfully")
print("=== CELL 2 COMPLETED SUCCESSFULLY ===\n")

=== CELL 2: UTILITIES AND GRID PRIMITIVES ===
WHO: Core system components
WHAT: Defining budget management, data loading, and grid operations
WHEN: During system initialization
WHERE: In memory for runtime operations
WHY: To handle ARC data and provide fundamental grid manipulation
HOW: Implementing classes for data management and grid processing
✓ All utility classes and functions defined successfully
=== CELL 2 COMPLETED SUCCESSFULLY ===



In [3]:
# Cell 3: Symbolic DSL Primitives
print("=== CELL 3: SYMBOLIC DSL PRIMITIVES ===")
print("WHO: Domain Specific Language definition")
print("WHAT: Implementing all grid transformation operations")
print("WHEN: During program synthesis and execution")
print("WHERE: In memory for symbolic reasoning")
print("WHY: To provide building blocks for solving ARC tasks")
print("HOW: Defining geometric, color, size, and object operations")

class DSL:
    # Geometric operations
    @staticmethod
    def identity(g: np.ndarray) -> np.ndarray: 
        print("✓ Applied identity transformation")
        return g.copy()
    
    @staticmethod
    def rotate_90(g: np.ndarray) -> np.ndarray: 
        print("✓ Applied 90° rotation")
        return np.rot90(g, 1)
    
    @staticmethod
    def rotate_180(g: np.ndarray) -> np.ndarray: 
        print("✓ Applied 180° rotation")
        return np.rot90(g, 2)
    
    @staticmethod
    def rotate_270(g: np.ndarray) -> np.ndarray: 
        print("✓ Applied 270° rotation")
        return np.rot90(g, 3)
    
    @staticmethod
    def flip_h(g: np.ndarray) -> np.ndarray: 
        print("✓ Applied horizontal flip")
        return np.fliplr(g)
    
    @staticmethod
    def flip_v(g: np.ndarray) -> np.ndarray: 
        print("✓ Applied vertical flip")
        return np.flipud(g)
    
    @staticmethod
    def transpose(g: np.ndarray) -> np.ndarray: 
        print("✓ Applied transpose")
        return g.T
    
    @staticmethod
    def reflect_diagonal(g: np.ndarray) -> np.ndarray: 
        print("✓ Applied diagonal reflection")
        return np.flip(g.T, axis=0)
    
    @staticmethod
    def reflect_anti_diagonal(g: np.ndarray) -> np.ndarray: 
        print("✓ Applied anti-diagonal reflection")
        return np.flip(g.T, axis=1)
    
    @staticmethod
    def shift_left(g: np.ndarray): 
        print("✓ Applied left shift")
        return np.roll(g, -1, axis=1)
    
    @staticmethod
    def shift_right(g: np.ndarray): 
        print("✓ Applied right shift")
        return np.roll(g, 1, axis=1)
    
    @staticmethod
    def shift_up(g: np.ndarray): 
        print("✓ Applied up shift")
        return np.roll(g, -1, axis=0)
    
    @staticmethod
    def shift_down(g: np.ndarray): 
        print("✓ Applied down shift")
        return np.roll(g, 1, axis=0)
    
    # Color operations
    @staticmethod
    def recolor_with_palette_bijection(g: np.ndarray, reference: np.ndarray) -> np.ndarray:
        if g.size == 0 or reference.size == 0 or len(np.unique(g)) <= 1 or len(np.unique(reference)) <= 1:
            return g.copy()
        
        colors_in = np.unique(g)
        colors_ref = np.unique(reference)
        cost = np.zeros((len(colors_in), len(colors_ref)))
        
        for i, cin in enumerate(colors_in):
            for j, cref in enumerate(colors_ref):
                mask_in = (g == cin)
                mask_ref = (reference == cref)
                cost[i, j] = mask_in.sum() - np.sum(mask_in & mask_ref)
        
        row_ind, col_ind = linear_sum_assignment(cost)
        color_map = {colors_in[i]: colors_ref[j] for i, j in zip(row_ind, col_ind) if j < len(colors_ref)}
        
        result = g.copy()
        for cin, cout in color_map.items():
            result[g == cin] = cout
        
        unmapped = set(colors_in) - set(color_map.keys())
        for cin in unmapped:
            result[g == cin] = CONFIG["BG"]
        
        print(f"✓ Applied palette bijection: {color_map}")
        return result.astype(np.uint8)
    
    @staticmethod
    def invert_colors(g: np.ndarray) -> np.ndarray: 
        inverted = np.where(g == 0, 0, 9 - g)
        print("✓ Applied color inversion")
        return inverted
    
    @staticmethod
    def replace_color(g: np.ndarray, old_color: int, new_color: int) -> np.ndarray:
        result = g.copy()
        result[g == old_color] = new_color
        print(f"✓ Replaced color {old_color} with {new_color}")
        return result
    
    @staticmethod
    def swap_most_common_colors(g: np.ndarray) -> np.ndarray:
        colors, counts = np.unique(g, return_counts=True)
        if len(colors) < 2: 
            print("✗ Not enough colors to swap")
            return g.copy()
        
        sorted_idx = np.argsort(-counts)
        c1, c2 = colors[sorted_idx[0]], colors[sorted_idx[1]]
        result = g.copy()
        result[g == c1] = c2
        result[g == c2] = c1
        print(f"✓ Swapped most common colors: {c1} <-> {c2}")
        return result
    
    @staticmethod
    def fill_mode_color(g: np.ndarray) -> np.ndarray:
        mode_color = Counter(g.flatten()).most_common(1)[0][0]
        filled = np.full_like(g, mode_color)
        print(f"✓ Filled with mode color: {mode_color}")
        return filled
    
    # Size operations
    @staticmethod
    def scale_2x(g: np.ndarray) -> np.ndarray: 
        scaled = np.repeat(np.repeat(g, 2, axis=0), 2, axis=1)
        print("✓ Applied 2x scaling")
        return scaled
    
    @staticmethod
    def scale_3x(g: np.ndarray) -> np.ndarray: 
        scaled = np.repeat(np.repeat(g, 3, axis=0), 3, axis=1)
        print("✓ Applied 3x scaling")
        return scaled
    
    @staticmethod
    def tile_2x2(g: np.ndarray) -> np.ndarray: 
        tiled = np.block([[g, g], [g, g]])
        print("✓ Applied 2x2 tiling")
        return tiled
    
    @staticmethod
    def tile_3x3(g: np.ndarray) -> np.ndarray: 
        tiled = np.block([[g, g, g], [g, g, g], [g, g, g]])
        print("✓ Applied 3x3 tiling")
        return tiled
    
    @staticmethod
    def crop_to_content(g: np.ndarray, bg=CONFIG["BG"]) -> np.ndarray:
        mask = g != bg
        if not mask.any(): 
            print("✗ No content to crop")
            return g.copy()
        
        rows = np.any(mask, axis=1)
        cols = np.any(mask, axis=0)
        cropped = g[rows][:, cols]
        print(f"✓ Cropped to content: {g.shape} -> {cropped.shape}")
        return cropped
    
    @staticmethod
    def add_border(g: np.ndarray, b=1, color=1) -> np.ndarray:
        h, w = g.shape
        result = np.full((h + 2*b, w + 2*b), color, dtype=g.dtype)
        result[b:h+b, b:w+b] = g
        print(f"✓ Added border of size {b} with color {color}")
        return result
    
    # Object operations
    @staticmethod
    def extract_largest_object(g: np.ndarray, bg_color: int = CONFIG["BG"]) -> np.ndarray:
        components = GridPrimitives.connected_components(g, bg_color)
        if not components: 
            print("✗ No objects to extract")
            return g.copy()
        
        largest = max(components, key=lambda x: x[0].sum())
        result = np.full_like(g, bg_color)
        result[largest[0]] = largest[1]
        print(f"✓ Extracted largest object (color {largest[1]}, size {largest[0].sum()})")
        return result
    
    @staticmethod
    def gravity_down(g: np.ndarray, bg=CONFIG["BG"]) -> np.ndarray:
        result = np.full_like(g, bg)
        for col in range(g.shape[1]):
            non_bg = g[:, col][g[:, col] != bg]
            if len(non_bg) > 0:
                result[-len(non_bg):, col] = non_bg
        print("✓ Applied gravity down")
        return result

# Define PRIMITIVE_FAMILIES
PRIMITIVE_FAMILIES = {
    'geometric': {
        'identity': DSL.identity,
        'rotate_90': DSL.rotate_90,
        'rotate_180': DSL.rotate_180,
        'rotate_270': DSL.rotate_270,
        'flip_h': DSL.flip_h,
        'flip_v': DSL.flip_v,
        'transpose': DSL.transpose,
        'reflect_diagonal': DSL.reflect_diagonal,
        'reflect_anti_diagonal': DSL.reflect_anti_diagonal,
        'shift_left': DSL.shift_left,
        'shift_right': DSL.shift_right,
        'shift_up': DSL.shift_up,
        'shift_down': DSL.shift_down,
    },
    'color': {
        'recolor_with_palette_bijection': lambda g: DSL.recolor_with_palette_bijection(g, g),
        'invert_colors': DSL.invert_colors,
        'replace_color_0_1': lambda g: DSL.replace_color(g, 0, 1),
        'replace_color_1_0': lambda g: DSL.replace_color(g, 1, 0),
        'swap_most_common_colors': DSL.swap_most_common_colors,
        'fill_mode_color': DSL.fill_mode_color,
    },
    'size': {
        'scale_2x': DSL.scale_2x,
        'scale_3x': DSL.scale_3x,
        'tile_2x2': DSL.tile_2x2,
        'tile_3x3': DSL.tile_3x3,
        'crop_to_content': DSL.crop_to_content,
        'add_border': DSL.add_border,
    },
    'object': {
        'extract_largest_object': DSL.extract_largest_object,
        'gravity_down': DSL.gravity_down,
        'gravity_up': lambda g: np.rot90(DSL.gravity_down(np.rot90(g, 2)), 2),
        'gravity_left': lambda g: np.rot90(DSL.gravity_down(np.rot90(g, 1)), 3),
        'gravity_right': lambda g: np.rot90(DSL.gravity_down(np.rot90(g, 3)), 1),
    }
}

print(f"✓ DSL defined with {sum(len(ops) for ops in PRIMITIVE_FAMILIES.values())} primitives")
print("✓ PRIMITIVE_FAMILIES organized by operation type")

# Validate all primitives
print("Validating all primitives...")
for family, ops in PRIMITIVE_FAMILIES.items():
    for name, func in ops.items():
        try:
            test_input = np.zeros((3, 3), dtype=np.uint8)
            test_input[1, 1] = 1  # Add a dot in the middle
            result = func(test_input)
            print(f"  ✓ {family}.{name}: SUCCESS")
        except Exception as e:
            print(f"  ✗ {family}.{name}: FAILED - {e}")

def generate_dynamic_replaces(train_colors):
    """Generate dynamic color replacement operations based on training colors"""
    dynamic_ops = {}
    colors = sorted(list(train_colors))
    print(f"Generating dynamic color replacements for colors: {colors}")
    
    if len(colors) >= 2:
        for i, c1 in enumerate(colors):
            for j, c2 in enumerate(colors):
                if i != j:
                    # Use closure to capture current values
                    dynamic_ops[f'replace_{c1}_with_{c2}'] = (lambda c1=c1, c2=c2: 
                        lambda g: DSL.replace_color(g, c1, c2))()
    
    print(f"✓ Generated {len(dynamic_ops)} dynamic color replacement operations")
    return dynamic_ops

print("=== CELL 3 COMPLETED SUCCESSFULLY ===\n")

=== CELL 3: SYMBOLIC DSL PRIMITIVES ===
WHO: Domain Specific Language definition
WHAT: Implementing all grid transformation operations
WHEN: During program synthesis and execution
WHERE: In memory for symbolic reasoning
WHY: To provide building blocks for solving ARC tasks
HOW: Defining geometric, color, size, and object operations
✓ DSL defined with 30 primitives
✓ PRIMITIVE_FAMILIES organized by operation type
Validating all primitives...
✓ Applied identity transformation
  ✓ geometric.identity: SUCCESS
✓ Applied 90° rotation
  ✓ geometric.rotate_90: SUCCESS
✓ Applied 180° rotation
  ✓ geometric.rotate_180: SUCCESS
✓ Applied 270° rotation
  ✓ geometric.rotate_270: SUCCESS
✓ Applied horizontal flip
  ✓ geometric.flip_h: SUCCESS
✓ Applied vertical flip
  ✓ geometric.flip_v: SUCCESS
✓ Applied transpose
  ✓ geometric.transpose: SUCCESS
✓ Applied diagonal reflection
  ✓ geometric.reflect_diagonal: SUCCESS
✓ Applied anti-diagonal reflection
  ✓ geometric.reflect_anti_diagonal: SUCCESS
✓ Ap

In [4]:
# Cell 4: Beam Search and Veto Network
print("=== CELL 4: BEAM SEARCH AND VETO NETWORK ===")
print("WHO: Program synthesis engine")
print("WHAT: Implementing beam search with neural guidance")
print("WHEN: During task solving phase")
print("WHERE: In memory for program search")
print("WHY: To efficiently explore program space with intelligent pruning")
print("HOW: Combining symbolic search with neural veto network")

@dataclass
class Node:
    grid: np.ndarray
    program: List[Tuple[str, str]] = field(default_factory=list)
    family_counts: Dict[str, int] = field(default_factory=dict)
    score: float = 0.0
    
    def __post_init__(self):
        print(f"✓ Node created: program_len={len(self.program)}, score={self.score:.3f}")

class TinyVetoNet(nn.Module):
    def __init__(self, input_dim=64, hidden_dim=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        print(f"✓ TinyVetoNet initialized: {input_dim} -> {hidden_dim} -> 1")
    
    def forward(self, x):
        result = torch.sigmoid(self.net(x)).squeeze(-1)
        print(f"✓ Veto network forward: input_shape={x.shape}, output_shape={result.shape}")
        return result

class ProgramCache:
    def __init__(self, capacity=CONFIG["CACHE_CAPACITY"]):
        self.cache = OrderedDict()
        self.capacity = capacity
        self.hits = 0
        self.misses = 0
        print(f"✓ ProgramCache initialized with capacity {capacity}")
    
    def get(self, task_id: str, program: list, input_grid: np.ndarray) -> Optional[np.ndarray]:
        key = (task_id, tuple(program), hash(input_grid.tobytes()))
        if key in self.cache:
            self.hits += 1
            self.cache.move_to_end(key)
            print(f"✓ Cache HIT: {key[0]}, program_len={len(program)}")
            return self.cache[key]
        self.misses += 1
        print(f"✗ Cache MISS: {key[0]}, program_len={len(program)}")
        return None
    
    def set(self, task_id: str, program: list, input_grid: np.ndarray, result: np.ndarray):
        key = (task_id, tuple(program), hash(input_grid.tobytes()))
        self.cache[key] = result
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)
        print(f"✓ Cache SET: {key[0]}, program_len={len(program)}, cache_size={len(self.cache)}")

class BeamSearchSolver:
    def __init__(self, veto_net: TinyVetoNet = None):
        self.veto_net = veto_net or TinyVetoNet()
        self.program_cache = ProgramCache()
        print("✓ BeamSearchSolver initialized with veto network and program cache")
    
    def solve(self, task: Dict, time_budget: float = CONFIG["TIME_BUDGET_S"]) -> Optional[Dict]:
        print(f"Starting beam search for task with {len(task['train'])} training examples")
        t0 = time.time()
        task_id = "task_" + str(hash(str(task)))[:8]
        
        # Dynamic color replacements
        train_colors = set()
        for pair in task["train"]:
            train_colors.update(np.unique(pair["input"]))
            train_colors.update(np.unique(pair["output"]))
        
        dynamic_replaces = generate_dynamic_replaces(train_colors)
        original_color_ops = PRIMITIVE_FAMILIES["color"].copy()
        PRIMITIVE_FAMILIES["color"].update(dynamic_replaces)
        print(f"✓ Added {len(dynamic_replaces)} dynamic color operations")
        
        # Initialize beam
        beam = [Node(np.array(task["test"][0]["input"]))]
        best_node = None
        best_score = 0.0
        
        print(f"Starting beam search with width {CONFIG['BEAM_WIDTH']}, depth {CONFIG['BEAM_DEPTH']}")
        
        for depth in range(CONFIG["BEAM_DEPTH"]):
            if time.time() - t0 > time_budget:
                print("✗ Time budget exceeded")
                break
                
            candidates = []
            print(f"Depth {depth}: Processing {len(beam)} nodes")
            
            for node_idx, node in enumerate(beam):
                for family, ops in PRIMITIVE_FAMILIES.items():
                    if node.family_counts.get(family, 0) >= CONFIG["FAMILY_CAPS"][family]:
                        continue
                        
                    for op_name, op_func in ops.items():
                        # Check cache
                        cached = self.program_cache.get(task_id, node.program + [(family, op_name)], node.grid)
                        
                        if cached is not None:
                            new_grid = cached
                            from_cache = True
                        else:
                            try:
                                new_grid = op_func(node.grid)
                                from_cache = False
                                self.program_cache.set(task_id, node.program + [(family, op_name)], node.grid, new_grid)
                            except Exception as e:
                                print(f"✗ Operation failed: {family}.{op_name} - {e}")
                                continue
                        
                        # Score the new node
                        new_node = Node(
                            new_grid, 
                            node.program + [(family, op_name)], 
                            dict(node.family_counts)
                        )
                        new_node.family_counts[family] = new_node.family_counts.get(family, 0) + 1
                        new_node.score = self.score_program(new_node.grid, task["train"])
                        
                        candidates.append(new_node)
                        
                        if new_node.score > best_score:
                            best_score = new_node.score
                            best_node = new_node
                            print(f"✓ New best score: {best_score:.3f} at depth {depth}")
            
            # Prune beam
            candidates.sort(key=lambda x: x.score, reverse=True)
            beam = candidates[:CONFIG["BEAM_WIDTH"]]
            print(f"Depth {depth}: Pruned to {len(beam)} candidates, best_score={best_score:.3f}")
            
            if best_score >= CONFIG["EARLY_STOP_CONF"]:
                print("✓ Early stopping condition met")
                break
        
        # Restore original color operations
        PRIMITIVE_FAMILIES["color"] = original_color_ops
        
        if best_node and best_score >= CONFIG["CONFIDENCE_THRESHOLD"]:
            print(f"✓ Solution found: score={best_score:.3f}, program_length={len(best_node.program)}")
            return {
                "output": best_node.grid.tolist(), 
                "confidence": best_score,
                "program": best_node.program
            }
        else:
            print("✗ No satisfactory solution found")
            return None
    
    def score_program(self, grid, train_pairs):
        if not train_pairs: 
            return 0.5
        
        scores = []
        for i, p in enumerate(train_pairs):
            tgt = np.array(p["output"])
            if grid.shape != tgt.shape:
                scores.append(0.0)
                continue
            
            accuracy = np.mean(grid == tgt)
            scores.append(accuracy)
        
        final_score = min(scores) if CONFIG["RELAXED_FIT_OK"] else sum(scores) / len(scores)
        print(f"✓ Program scored: {final_score:.3f} based on {len(train_pairs)} examples")
        return final_score

print("✓ Beam search system fully implemented")
print("=== CELL 4 COMPLETED SUCCESSFULLY ===\n")

=== CELL 4: BEAM SEARCH AND VETO NETWORK ===
WHO: Program synthesis engine
WHAT: Implementing beam search with neural guidance
WHEN: During task solving phase
WHERE: In memory for program search
WHY: To efficiently explore program space with intelligent pruning
HOW: Combining symbolic search with neural veto network
✓ Beam search system fully implemented
=== CELL 4 COMPLETED SUCCESSFULLY ===



In [5]:
# Cell 5: Neural Components (UPDATED VERSION)
print("=== CELL 5: NEURAL COMPONENTS ===")
print("WHO: Deep learning subsystems")
print("WHAT: Implementing neural networks for program learning")
print("WHEN: During training and neural-based solving")
print("WHERE: On GPU/CPU for neural inference")
print("WHY: To complement symbolic search with learned patterns")
print("HOW: Using VAEs, transformers, and policy networks")

class PlatonicVAE(nn.Module):
    def __init__(self, in_channels=1, latent_dim=CONFIG["LATENT_DIM"]):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1), nn.ReLU(),
            nn.Flatten(), nn.Linear(32*30*30, latent_dim*2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32*30*30), nn.ReLU(),
            nn.Unflatten(1, (32, 30, 30)), nn.ConvTranspose2d(32, 1, 3, padding=1)
        )
        print(f"✓ PlatonicVAE initialized: {in_channels}->{latent_dim}->{in_channels}")

    def forward(self, x):
        mu_logvar = self.encoder(x)
        mu, logvar = mu_logvar.chunk(2, dim=1)
        z = mu + torch.exp(0.5 * logvar) * torch.randn_like(mu)
        reconstructed = self.decoder(z).squeeze(1)
        print(f"✓ VAE forward: {x.shape} -> {reconstructed.shape}")
        return reconstructed, mu, logvar

class NeuralPolicySolver(nn.Module):
    def __init__(self):
        super().__init__()
        self.vocab = []
        for ops in PRIMITIVE_FAMILIES.values():
            self.vocab.extend(list(ops.keys()))
        self.vocab = ["EOS"] + self.vocab
        self.vocab_size = len(self.vocab)
        
        self.policy_net = nn.Sequential(
            nn.Linear(128, 256), nn.ReLU(),
            nn.Linear(256, 128), nn.ReLU(), 
            nn.Linear(128, self.vocab_size)
        )
        print(f"✓ NeuralPolicySolver initialized with vocab size {self.vocab_size}")

    def solve(self, task: Dict):
        print("Starting neural policy solving...")
        
        # Always return a properly structured result
        try:
            test_input = np.array(task["test"][0]["input"])
            input_tensor = torch.tensor(test_input.flatten()[:128]).float().unsqueeze(0)
            
            # Pad or truncate to 128 features
            if input_tensor.shape[1] < 128:
                input_tensor = F.pad(input_tensor, (0, 128 - input_tensor.shape[1]))
            elif input_tensor.shape[1] > 128:
                input_tensor = input_tensor[:, :128]
            
            logits = self.policy_net(input_tensor)
            probs = F.softmax(logits, dim=-1)
            
            # Simple strategy: try a few common operations
            result_grid = test_input.copy()
            confidence = 0.3  # Base confidence for neural approach
            
            # Try some simple transformations
            if test_input.shape[0] > 1 and test_input.shape[1] > 1:
                try:
                    # Try rotation if it seems helpful
                    if np.mean(test_input == 0) > 0.8:  # Mostly background
                        result_grid = DSL.rotate_90(test_input)
                        confidence = 0.5
                    # Try color inversion
                    elif len(np.unique(test_input)) > 2:
                        result_grid = DSL.invert_colors(test_input)
                        confidence = 0.4
                except Exception as e:
                    print(f"✗ Neural transformation failed: {e}")
            
            print(f"✓ Neural policy completed with confidence {confidence:.3f}")
            
            # Return properly structured result
            return {
                "output": result_grid.tolist(),
                "confidence": confidence,
                "method": "neural"
            }
            
        except Exception as e:
            print(f"✗ Neural solver failed: {e}")
            # Return fallback result
            test_input = task["test"][0]["input"]
            return {
                "output": test_input,
                "confidence": 0.1,
                "method": "neural_fallback"
            }

print("✓ Neural components defined and ready")
print("=== CELL 5 COMPLETED SUCCESSFULLY ===\n")

=== CELL 5: NEURAL COMPONENTS ===
WHO: Deep learning subsystems
WHAT: Implementing neural networks for program learning
WHEN: During training and neural-based solving
WHERE: On GPU/CPU for neural inference
WHY: To complement symbolic search with learned patterns
HOW: Using VAEs, transformers, and policy networks
✓ Neural components defined and ready
=== CELL 5 COMPLETED SUCCESSFULLY ===



In [6]:
# Cell 6: Hybrid Solver and Main System (FIXED VERSION)
print("=== CELL 6: HYBRID SOLVER AND MAIN SYSTEM ===")
print("WHO: Main orchestrator")
print("WHAT: Combining symbolic and neural approaches")
print("WHEN: During final task solving")
print("WHERE: Coordinating between all subsystems")
print("WHY: To leverage strengths of both symbolic and neural methods")
print("HOW: Fallback strategy with confidence-based selection")

class UltimateDamageARCSolverAGI(nn.Module):
    def __init__(self):
        super().__init__()
        self.symbolic_solver = BeamSearchSolver(TinyVetoNet())
        self.neural_solver = NeuralPolicySolver()
        print("✓ UltimateDamageARCSolverAGI initialized with both solvers")

    def solve_task(self, task: Dict) -> List[Dict]:
        print(f"\n{'='*50}")
        print(f"SOLVING TASK WITH {len(task['train'])} TRAIN EXAMPLES")
        print(f"{'='*50}")
        
        attempts = []
        
        # Attempt 1: Symbolic beam search
        print("\n--- ATTEMPT 1: SYMBOLIC BEAM SEARCH ---")
        symbolic_result = self.symbolic_solver.solve(task)
        if symbolic_result:
            # Ensure symbolic result has method field
            symbolic_result["method"] = symbolic_result.get("method", "symbolic")
            attempts.append(symbolic_result)
            print(f"✓ Symbolic attempt: confidence {symbolic_result['confidence']:.3f}")
        else:
            print("✗ Symbolic attempt failed")
        
        # Attempt 2: Neural policy
        print("\n--- ATTEMPT 2: NEURAL POLICY ---")
        neural_result = self.neural_solver.solve(task)
        if neural_result:
            # Neural result should already have method from Cell 5
            attempts.append(neural_result)
            print(f"✓ Neural attempt: confidence {neural_result['confidence']:.3f}")
        else:
            print("✗ Neural attempt failed")
        
        # Ensure we have at least 2 attempts
        if len(attempts) < 2:
            print("\n--- ATTEMPT 3: FALLBACK STRATEGY ---")
            test_input = task["test"][0]["input"]
            fallback_result = {
                "output": test_input,  # Identity transformation
                "confidence": 0.1,
                "method": "fallback"
            }
            attempts.append(fallback_result)
            print("✓ Fallback attempt: identity transformation")
        
        # Ensure all attempts have required fields
        for i, attempt in enumerate(attempts):
            if "method" not in attempt:
                attempt["method"] = f"unknown_{i}"
            if "confidence" not in attempt:
                attempt["confidence"] = 0.0
            if "output" not in attempt:
                attempt["output"] = task["test"][0]["input"]
        
        # Sort by confidence
        attempts.sort(key=lambda x: x["confidence"], reverse=True)
        
        # Safe formatting of attempt descriptions
        attempt_descriptions = []
        for a in attempts:
            method = a.get("method", "unknown")
            confidence = a.get("confidence", 0.0)
            desc = f"{method}({confidence:.3f})"
            attempt_descriptions.append(desc)
        
        print(f"\n✓ Final attempts: {attempt_descriptions}")
        
        return attempts

class HybridSolver:
    def __init__(self):
        self.model = UltimateDamageARCSolverAGI()
        print("✓ HybridSolver initialized")

    def solve(self, tasks: Dict) -> Dict:
        print(f"Starting batch solving of {len(tasks)} tasks")
        submissions = {}
        
        for task_id, task in tasks.items():
            print(f"\nProcessing task {task_id}...")
            
            # Validate task structure
            if "test" not in task or len(task["test"]) == 0:
                print(f"✗ Task {task_id} has no test examples, skipping")
                continue
                
            if "input" not in task["test"][0]:
                print(f"✗ Task {task_id} test has no input, skipping")
                continue
            
            try:
                attempts = self.model.solve_task(task)
                
                # Ensure we have at least 2 attempts for submission format
                if len(attempts) < 2:
                    print(f"⚠ Only {len(attempts)} attempts, adding fallbacks")
                    test_input = task["test"][0]["input"]
                    while len(attempts) < 2:
                        fallback = {
                            "output": test_input,
                            "confidence": 0.01,
                            "method": f"fallback_{len(attempts)+1}"
                        }
                        attempts.append(fallback)
                
                # Format for submission
                submission_entry = {
                    "attempt_1": attempts[0]["output"],
                    "attempt_2": attempts[1]["output"]
                }
                submissions[task_id] = [submission_entry]
                
                print(f"✓ Task {task_id} completed: {len(attempts)} attempts")
                
            except Exception as e:
                print(f"✗ Error solving task {task_id}: {e}")
                # Create fallback submission for this task
                test_input = task["test"][0]["input"] if "test" in task and len(task["test"]) > 0 else [[0]]
                submission_entry = {
                    "attempt_1": test_input,
                    "attempt_2": test_input
                }
                submissions[task_id] = [submission_entry]
                print(f"✓ Created fallback submission for task {task_id}")
        
        print(f"✓ Batch solving completed: {len(submissions)} tasks processed")
        return submissions

print("✓ Hybrid solving system ready")
print("=== CELL 6 COMPLETED SUCCESSFULLY ===\n")

=== CELL 6: HYBRID SOLVER AND MAIN SYSTEM ===
WHO: Main orchestrator
WHAT: Combining symbolic and neural approaches
WHEN: During final task solving
WHERE: Coordinating between all subsystems
WHY: To leverage strengths of both symbolic and neural methods
HOW: Fallback strategy with confidence-based selection
✓ Hybrid solving system ready
=== CELL 6 COMPLETED SUCCESSFULLY ===



In [7]:
# Cell 7: Dataset and Training Infrastructure
print("=== CELL 7: DATASET AND TRAINING INFRASTRUCTURE ===")
print("WHO: Data management system")
print("WHAT: Handling ARC dataset loading and processing")
print("WHEN: During training and evaluation phases")
print("WHERE: Loading from disk to memory")
print("WHY: To provide structured access to ARC challenges")
print("HOW: Using PyTorch Dataset and DataLoader")

class ARCDataset(Dataset):
    def __init__(self, data_path, solutions_path=None, mode='train'):
        print(f"Loading ARC dataset from {data_path} (mode: {mode})")
        
        with open(data_path, 'r') as f:
            self.data = json.load(f)
        
        self.tasks = list(self.data.items())
        self.mode = mode
        self.solutions = None
        
        if solutions_path and mode == 'train' and os.path.exists(solutions_path):
            with open(solutions_path, 'r') as f:
                self.solutions = json.load(f)
            print(f"✓ Loaded solutions from {solutions_path}")
        
        print(f"✓ Dataset initialized: {len(self.tasks)} tasks, mode={mode}")
    
    def __len__(self):
        return len(self.tasks)
    
    def __getitem__(self, idx):
        tid, task = self.tasks[idx]
        
        # Process training pairs
        train_pairs = []
        for p in task.get('train', []):
            try:
                input_arr = np.array(p['input'])
                output_arr = np.array(p['output']) if self.solutions else None
                train_pairs.append({'input': input_arr, 'output': output_arr})
            except Exception as e:
                print(f"✗ Error processing train pair: {e}")
                continue
        
        # Process test inputs
        test_inputs = []
        for p in task.get('test', []):
            try:
                test_inputs.append(np.array(p['input']))
            except Exception as e:
                print(f"✗ Error processing test input: {e}")
                continue
        
        solution = self.solutions.get(tid) if self.solutions else None
        
        print(f"✓ Loaded task {tid}: {len(train_pairs)} train, {len(test_inputs)} test")
        return tid, train_pairs, test_inputs, solution

def train_model(model, dataset, epochs=3):
    print(f"Starting training for {epochs} epochs on {len(dataset)} tasks")
    
    # Simple training loop - in practice, this would be more sophisticated
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    
    for epoch in range(epochs):
        epoch_loss = 0
        print(f"Epoch {epoch+1}/{epochs}")
        
        for i, (tid, train_pairs, test_inputs, _) in enumerate(dataset):
            if i >= 10:  # Limit for demonstration
                break
                
            # Simple training: try to learn from input-output pairs
            for pair in train_pairs:
                if pair['output'] is not None:
                    try:
                        # Convert to tensor
                        input_tensor = torch.tensor(pair['input']).float().unsqueeze(0).unsqueeze(0)
                        target_tensor = torch.tensor(pair['output']).float().unsqueeze(0).unsqueeze(0)
                        
                        # Forward pass through VAE
                        reconstructed, mu, logvar = model.symbolic_solver.veto_net.left_induction(input_tensor)
                        loss = F.mse_loss(reconstructed, target_tensor)
                        
                        # Backward pass
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        
                        epoch_loss += loss.item()
                        
                    except Exception as e:
                        print(f"✗ Training step failed: {e}")
                        continue
        
        print(f"✓ Epoch {epoch+1} completed: avg_loss={epoch_loss/len(dataset):.4f}")
    
    print("✓ Training completed")

print("✓ Dataset and training infrastructure ready")
print("=== CELL 7 COMPLETED SUCCESSFULLY ===\n")

=== CELL 7: DATASET AND TRAINING INFRASTRUCTURE ===
WHO: Data management system
WHAT: Handling ARC dataset loading and processing
WHEN: During training and evaluation phases
WHERE: Loading from disk to memory
WHY: To provide structured access to ARC challenges
HOW: Using PyTorch Dataset and DataLoader
✓ Dataset and training infrastructure ready
=== CELL 7 COMPLETED SUCCESSFULLY ===



In [8]:
# Cell 8: Main Execution and Submission (FINAL - TRULY SILENT VERSION)
print("=== CELL 8: MAIN EXECUTION AND SUBMISSION ===")
print("WHO: Main execution controller")
print("WHAT: Orchestrating the entire ARC solving pipeline")
print("WHEN: At notebook runtime")
print("WHERE: Coordinating all subsystems")
print("WHY: To generate final submission for ARC Prize 2025")
print("HOW: Loading real competition data and solving tasks")

# Add competition speed optimizations
COMPETITION_CONFIG = {
    "BEAM_WIDTH": 8,           # Reduce from 16
    "BEAM_DEPTH": 2,           # Reduce from 6
    "TIME_BUDGET_S": 8,        # Reduce from 30
    "FAMILY_CAPS": {"geometric": 1, "color": 2, "size": 1, "object": 1},  # Reduce caps
}

# Apply config overrides
for key, value in COMPETITION_CONFIG.items():
    CONFIG[key] = value

print("✓ Using competition-optimized config for speed")

class SilentProgramCache(ProgramCache):
    """Completely silent program cache"""
    def get(self, task_id: str, program: list, input_grid: np.ndarray) -> Optional[np.ndarray]:
        key = (task_id, tuple(program), hash(input_grid.tobytes()))
        if key in self.cache:
            self.hits += 1
            self.cache.move_to_end(key)
            return self.cache[key]
        self.misses += 1
        return None
    
    def set(self, task_id: str, program: list, input_grid: np.ndarray, result: np.ndarray):
        key = (task_id, tuple(program), hash(input_grid.tobytes()))
        self.cache[key] = result
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)

class SilentNode:
    """Silent version of Node without print spam"""
    def __init__(self, grid: np.ndarray, program: List[Tuple[str, str]] = None, 
                 family_counts: Dict[str, int] = None, score: float = 0.0):
        self.grid = grid
        self.program = program or []
        self.family_counts = family_counts or {}
        self.score = score

class UltraQuietBeamSolver(BeamSearchSolver):
    """Truly silent beam solver - overrides all noisy methods"""
    def __init__(self, veto_net: TinyVetoNet = None):
        self.veto_net = veto_net or TinyVetoNet()
        self.program_cache = SilentProgramCache()  # Use silent cache
    
    def solve(self, task: Dict, time_budget: float = CONFIG["TIME_BUDGET_S"]) -> Optional[Dict]:
        t0 = time.time()
        task_id = "task_" + str(hash(str(task)))[:8]
        
        # Dynamic color replacements (silent)
        train_colors = set()
        for pair in task["train"]:
            train_colors.update(np.unique(pair["input"]))
            train_colors.update(np.unique(pair["output"]))
        
        dynamic_replaces = generate_dynamic_replaces(train_colors)
        original_color_ops = PRIMITIVE_FAMILIES["color"].copy()
        PRIMITIVE_FAMILIES["color"].update(dynamic_replaces)
        
        # Initialize beam with silent nodes
        beam = [SilentNode(np.array(task["test"][0]["input"]))]
        best_node = None
        best_score = 0.0
        
        for depth in range(CONFIG["BEAM_DEPTH"]):
            if time.time() - t0 > time_budget:
                break
                
            candidates = []
            
            for node in beam:
                for family, ops in PRIMITIVE_FAMILIES.items():
                    if node.family_counts.get(family, 0) >= CONFIG["FAMILY_CAPS"][family]:
                        continue
                        
                    for op_name, op_func in ops.items():
                        # Check cache silently
                        cached = self.program_cache.get(task_id, node.program + [(family, op_name)], node.grid)
                        
                        if cached is not None:
                            new_grid = cached
                        else:
                            try:
                                new_grid = op_func(node.grid)
                                self.program_cache.set(task_id, node.program + [(family, op_name)], node.grid, new_grid)
                            except Exception:
                                continue
                        
                        # Score the new node
                        new_node_score = self.score_program_silent(new_grid, task["train"])
                        if new_node_score > 0.01:  # Only consider promising nodes
                            new_node = SilentNode(
                                new_grid, 
                                node.program + [(family, op_name)], 
                                dict(node.family_counts),
                                new_node_score
                            )
                            new_node.family_counts[family] = new_node.family_counts.get(family, 0) + 1
                            candidates.append(new_node)
                            
                            if new_node_score > best_score:
                                best_score = new_node_score
                                best_node = new_node
            
            # Prune beam
            if candidates:
                candidates.sort(key=lambda x: x.score, reverse=True)
                beam = candidates[:CONFIG["BEAM_WIDTH"]]
            else:
                break
            
            if best_score >= 0.9:  # Early stop for excellent solutions
                break
        
        # Restore original color operations
        PRIMITIVE_FAMILIES["color"] = original_color_ops
        
        if best_node and best_score >= 0.1:  # Lower threshold for competition
            return {
                "output": best_node.grid.tolist(), 
                "confidence": best_score,
                "method": "symbolic"
            }
        return None
    
    def score_program_silent(self, grid, train_pairs):
        """Silent scoring function"""
        if not train_pairs: 
            return 0.0
        
        min_score = 1.0
        for p in train_pairs:
            tgt = np.array(p["output"])
            if grid.shape != tgt.shape:
                return 0.0  # Shape mismatch = immediate fail
            
            accuracy = np.mean(grid == tgt)
            min_score = min(min_score, accuracy)
            
            # Early exit if any training pair fails completely
            if min_score == 0.0:
                break
        
        return min_score

class CompetitionSolver:
    """Solver optimized for ARC Prize 2025 competition"""
    def __init__(self):
        self.symbolic_solver = UltraQuietBeamSolver(TinyVetoNet())
        self.neural_solver = NeuralPolicySolver()
        print("✓ Competition solver initialized")
    
    def solve_task(self, task: Dict) -> List[Dict]:
        """Solve a single ARC task with multiple approaches"""
        attempts = []
        
        # Method 1: Fast pattern matching (primary - very fast)
        pattern_result = self.fast_pattern_match(task)
        if pattern_result:
            attempts.append(pattern_result)
        
        # Method 2: Symbolic beam search (secondary - slower but more thorough)
        if len(attempts) < 2:  # Only run if we need more attempts
            try:
                symbolic_result = self.symbolic_solver.solve(task)
                if symbolic_result and symbolic_result["confidence"] > 0.1:
                    attempts.append(symbolic_result)
            except Exception:
                pass
        
        # Method 3: Simple transformations (fallback - very fast)
        simple_result = self.simple_transformations(task)
        if simple_result:
            attempts.append(simple_result)
        
        # Ensure we have at least 2 attempts
        test_input = task["test"][0]["input"]
        while len(attempts) < 2:
            fallback = {
                "output": test_input,
                "confidence": 0.01,
                "method": f"fallback_{len(attempts)+1}"
            }
            attempts.append(fallback)
        
        # Sort by confidence and return top 2
        attempts.sort(key=lambda x: x["confidence"], reverse=True)
        return attempts[:2]
    
    def fast_pattern_match(self, task: Dict) -> Optional[Dict]:
        """Very fast pattern matching for common ARC patterns"""
        try:
            train_pairs = task["train"]
            test_input = np.array(task["test"][0]["input"])
            
            if not train_pairs:
                return None
            
            # Check for identical input/output (copy pattern)
            all_identical = True
            for pair in train_pairs:
                train_in = np.array(pair["input"])
                train_out = np.array(pair["output"])
                if not np.array_equal(train_in, train_out):
                    all_identical = False
                    break
            
            if all_identical:
                return {
                    "output": test_input.tolist(),
                    "confidence": 0.95,
                    "method": "copy_pattern"
                }
            
            # Check single training pair patterns (very fast)
            if len(train_pairs) == 1:
                train_input = np.array(train_pairs[0]["input"])
                train_output = np.array(train_pairs[0]["output"])
                
                # Check if shapes match for transformations
                if train_input.shape == train_output.shape:
                    # Check color mapping
                    color_map = {}
                    valid_map = True
                    for old, new in zip(train_input.flatten(), train_output.flatten()):
                        if old in color_map and color_map[old] != new:
                            valid_map = False
                            break
                        color_map[old] = new
                    
                    if valid_map and len(color_map) > 1:  # Only if mapping is non-trivial
                        result = test_input.copy()
                        for old_color, new_color in color_map.items():
                            result[test_input == old_color] = new_color
                        return {
                            "output": result.tolist(),
                            "confidence": 0.85,
                            "method": "color_map"
                        }
                
                # Check rotations (fast checks)
                if train_input.shape == train_output.shape:
                    for k in [1, 2, 3]:  # 90, 180, 270 degrees
                        if np.array_equal(np.rot90(train_input, k), train_output):
                            result = np.rot90(test_input, k)
                            return {
                                "output": result.tolist(),
                                "confidence": 0.9,
                                "method": f"rotate_{k*90}"
                            }
                
                # Check flips
                if train_input.shape == train_output.shape:
                    if np.array_equal(np.fliplr(train_input), train_output):
                        return {
                            "output": np.fliplr(test_input).tolist(),
                            "confidence": 0.9,
                            "method": "flip_lr"
                        }
                    if np.array_equal(np.flipud(train_input), train_output):
                        return {
                            "output": np.flipud(test_input).tolist(),
                            "confidence": 0.9,
                            "method": "flip_ud"
                        }
            
            return None
            
        except Exception:
            return None
    
    def simple_transformations(self, task: Dict) -> Optional[Dict]:
        """Apply simple transformations as fallback"""
        try:
            test_input = np.array(task["test"][0]["input"])
            
            # Try a few simple operations
            operations = [
                (DSL.rotate_90, "rotate_90", 0.3),
                (DSL.flip_lr, "flip_lr", 0.3),
                (DSL.invert_colors, "invert", 0.2),
            ]
            
            for op_func, op_name, confidence in operations:
                try:
                    result = op_func(test_input)
                    # Only return if transformation changed something
                    if not np.array_equal(result, test_input):
                        return {
                            "output": result.tolist(),
                            "confidence": confidence,
                            "method": op_name
                        }
                except Exception:
                    continue
            
            return None
            
        except Exception:
            return None

# [Rest of the file remains the same - load_arc_prize_2025_data(), load_test_challenges(), etc.]
def load_arc_prize_2025_data():
    """Load the actual ARC Prize 2025 dataset"""
    data_root = "/kaggle/input/arc-prize-2025"
    print(f"Loading ARC Prize 2025 data from: {data_root}")
    
    if not os.path.exists(data_root):
        print("✗ ARC Prize 2025 data directory not found!")
        return None, []
    
    # List available files
    available_files = os.listdir(data_root)
    print("Available files:")
    for file in available_files:
        file_path = os.path.join(data_root, file)
        file_size = os.path.getsize(file_path) if os.path.exists(file_path) else 0
        print(f"  ✓ {file} ({file_size / 1024:.1f} KB)")
    
    return data_root, available_files

def load_test_challenges(data_root):
    """Load the test challenges for submission"""
    test_file = "arc-agi_test-challenges.json"
    test_path = os.path.join(data_root, test_file)
    
    if not os.path.exists(test_path):
        print(f"✗ Test file not found: {test_file}")
        print("Trying alternative file names...")
        
        # Try alternative file names
        alternatives = [
            "arc-agi_test_challenges.json",
            "test-challenges.json", 
            "test_challenges.json",
            "arc-agi_evaluation-challenges.json",
            "evaluation-challenges.json"
        ]
        
        for alt_file in alternatives:
            alt_path = os.path.join(data_root, alt_file)
            if os.path.exists(alt_path):
                test_path = alt_path
                test_file = alt_file
                print(f"✓ Found alternative: {test_file}")
                break
    
    if not os.path.exists(test_path):
        print("✗ No test challenges file found")
        return {}
    
    try:
        with open(test_path, 'r') as f:
            test_data = json.load(f)
        print(f"✓ Loaded test challenges: {len(test_data)} tasks from {test_file}")
        return test_data
    except Exception as e:
        print(f"✗ Error loading test challenges: {e}")
        return {}

def main():
    print("🚀 ARC PRIZE 2025 SOLVER - ULTRA QUIET VERSION")
    print("=" * 60)
    
    # Load competition data
    print("\n--- LOADING COMPETITION DATA ---")
    data_root, available_files = load_arc_prize_2025_data()
    
    if not data_root:
        print("✗ Cannot proceed without data directory")
        return
    
    # Load test challenges for submission
    test_tasks = load_test_challenges(data_root)
    
    if not test_tasks:
        print("✗ No test tasks loaded. Cannot generate submission.")
        return
    
    print(f"✓ Loaded {len(test_tasks)} test tasks for submission")
    
    # Initialize competition solver
    print("\n--- INITIALIZING SOLVER ---")
    solver = CompetitionSolver()
    print("✓ Competition solver ready")
    
    # Solve tasks
    print("\n--- SOLVING TASKS ---")
    print(f"Solving {len(test_tasks)} tasks (completely silent mode)...")
    start_time = time.time()
    
    submissions = {}
    
    for i, (task_id, task) in enumerate(test_tasks.items()):
        # Simple progress indicator without spam
        if i % 50 == 0 and i > 0:
            elapsed = time.time() - start_time
            rate = i / elapsed
            print(f"  Progress: {i}/{len(test_tasks)} tasks ({rate:.1f} tasks/sec)")
        
        try:
            attempts = solver.solve_task(task)
            submission_entry = {
                "attempt_1": attempts[0]["output"],
                "attempt_2": attempts[1]["output"]
            }
            submissions[task_id] = [submission_entry]
            
        except Exception:
            # Silent fallback
            test_input = task["test"][0]["input"]
            submission_entry = {
                "attempt_1": test_input,
                "attempt_2": test_input
            }
            submissions[task_id] = [submission_entry]
    
    total_time = time.time() - start_time
    print(f"✓ Solving completed: {len(submissions)}/{len(test_tasks)} tasks in {total_time:.1f}s")
    print(f"✓ Average rate: {len(submissions)/total_time:.2f} tasks/second")
    
    # Generate submission file
    submission_path = "/kaggle/working/submission.json"
    with open(submission_path, 'w') as f:
        json.dump(submissions, f, indent=2)
    
    print(f"✓ Submission saved to: {submission_path}")
    print("\n🎯 SUBMISSION GENERATED SUCCESSFULLY!")
    print("=" * 60)

if __name__ == '__main__':
    main()

print("=== CELL 8 COMPLETED SUCCESSFULLY ===\n")

=== CELL 8: MAIN EXECUTION AND SUBMISSION ===
WHO: Main execution controller
WHAT: Orchestrating the entire ARC solving pipeline
WHEN: At notebook runtime
WHERE: Coordinating all subsystems
WHY: To generate final submission for ARC Prize 2025
HOW: Loading real competition data and solving tasks
✓ Using competition-optimized config for speed
🚀 ARC PRIZE 2025 SOLVER - ULTRA QUIET VERSION

--- LOADING COMPETITION DATA ---
Loading ARC Prize 2025 data from: /kaggle/input/arc-prize-2025
Available files:
  ✓ arc-agi_training_solutions.json (643.3 KB)
  ✓ arc-agi_evaluation_solutions.json (218.6 KB)
  ✓ arc-agi_evaluation_challenges.json (961.6 KB)
  ✓ sample_submission.json (19.5 KB)
  ✓ arc-agi_training_challenges.json (3916.1 KB)
  ✓ arc-agi_test_challenges.json (991.5 KB)
✗ Test file not found: arc-agi_test-challenges.json
Trying alternative file names...
✓ Found alternative: arc-agi_test_challenges.json
✓ Loaded test challenges: 240 tasks from arc-agi_test_challenges.json
✓ Loaded 240 tes